In [1]:
from requests import get
wiki = get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [2]:
from bs4 import BeautifulSoup as soup
parsed_wiki = soup(wiki.text, "lxml")
table_rows = parsed_wiki.body.table.tbody.find_all("tr")

In [3]:
import pandas as pd
postal_code_df = pd.read_html(wiki.text, na_values=["Not assigned"])
postal_code_df = postal_code_df[0]
postal_code_df.head(10)

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,NaN,NaN
2,M2A,NaN,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,NaN


In [4]:
postal_code_df2 = postal_code_df.drop([0], axis=0)
postal_code_df2.columns = ["Postcode", "Borough", "Neighbourhood"]
postal_code_df2.head()

,Postcode,Borough,Neighbourhood
1,M1A,NaN,NaN
2,M2A,NaN,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [5]:
postal_code_df2["Neighbourhood"].fillna(value=postal_code_df2["Borough"], inplace=True)
postal_code_df2.head(10)

,Postcode,Borough,Neighbourhood
1,M1A,NaN,NaN
2,M2A,NaN,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
10,M8A,NaN,NaN


In [6]:
postal_code_df2.dropna(inplace=True)
postal_code_df2.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [7]:
grouped_df = postal_code_df2.groupby(["Postcode", "Borough"]).aggregate(", ".join).reset_index()
grouped_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
grouped_df.shape

(103, 3)

In [10]:
! pip install geocoder

In [11]:
import geocoder

def get_geo_coordinates(postal_code):
    print("Looking up latitude and longitude of: {}".format(postal_code))
    lat_lng_coords = None

    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
    return (lat_lng_coords[0], lat_lng_coords[1])

In [12]:
geospatial_coord_df = pd.read_csv("https://cocl.us/Geospatial_data")
grouped_df["Postal Code"] = grouped_df["Postcode"]
grouped_df.drop("Postcode",axis=1, inplace=True)

In [13]:
postal_code_with_coords_df = grouped_df.merge(geospatial_coord_df, on="Postal Code")
postal_code_with_coords_df.head(10)

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848
